# Model Comparison Without Pipeline

This notebook compares different machine learning models using manual preprocessing and hyperparameter optimization with Optuna.

## Models Tested
- **LightGBM**: Gradient boosting framework
- **XGBoost**: Extreme gradient boosting
- **Random Forest**: Ensemble of decision trees  
- **H2O AutoML**: Automated machine learning framework

## Features
- Manual preprocessing with custom feature engineering
- Optuna hyperparameter optimization
- Model comparison and selection
- Performance evaluation across different algorithms

In [1]:
import utils
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from scipy.sparse import hstack
import pandas as pd
from skrub import TableReport, TableVectorizer
from jours_feries_france import JoursFeries
from vacances_scolaires_france import SchoolHolidayDates
import numpy as np
import joblib

In [3]:
X, y, X_test = utils.get_and_process_data()

/Users/felix/Python_Data_Challenge-1/utils.py:217: FutureWarning: The behavior of Series.argmax/argmin with skipna=False and NAs, or with all-NAs is deprecated. In a future version this will raise ValueError.
  (non_nan_values - value).abs().argmin()
/Users/felix/Python_Data_Challenge-1/utils.py:217: FutureWarning: The behavior of Series.argmax/argmin with skipna=False and NAs, or with all-NAs is deprecated. In a future version this will raise ValueError.
  (non_nan_values - value).abs().argmin()
/Users/felix/Python_Data_Challenge-1/utils.py:383: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting val

In [ ]:
# Split the data into training and validation sets based on the last 10% of dates
validation_split_index = int(len(X) * 0.9)
X_train, X_val = X.iloc[:validation_split_index], X.iloc[validation_split_index:]
y_train, y_val = y.iloc[:validation_split_index], y.iloc[validation_split_index:]

# Initialize the TableVectorizer
vectorizer = TableVectorizer()

# Fit and transform the training data
X_train_transformed = vectorizer.fit_transform(X_train)
X_val_transformed = vectorizer.transform(X_val)


/users/eleves-a/2024/felix.brochier/Python_Data_Challenge-1/.venv/lib64/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'df' is not defined

Used the GPU of Ecole Polytechnique to accelerate the tuning of certain models such as XGBoost or LightGBM.

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import VotingRegressor, RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.pipeline import Pipeline

# Define GPU-enabled regressors
xgb_regressor = XGBRegressor(tree_method='gpu_hist', gpu_id=0, random_state=42)
lgbm_regressor = LGBMRegressor(device='gpu', gpu_device_id=0, random_state=42)

# Define CPU-based Random Forest
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)

# Columns to exclude from scaling
exclude_columns = ["is_weekend", "is_school_holiday", "road_work", "confinement", "couvre_feu"]
numerical_features = [
    col for col in X.columns if col not in exclude_columns + ["counter_name"]
]

# Preprocessing: One-Hot Encode "counter_name" and scale numerical columns
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),  # Scale numerical columns
        ('cat', OneHotEncoder(), ['counter_name'])      # One-hot encode "counter_name"
    ],
    remainder='passthrough'  # Keep other columns unchanged
)

# Define the Voting Regressor
voting_regressor = VotingRegressor([
    ('xgb', xgb_regressor),
    ('lgbm', lgbm_regressor),
    ('rf', rf_regressor)
])

# Complete Pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),  # Preprocessing step
    ('voting_regressor', voting_regressor)  # Voting regressor
])

# # Fit the pipeline
pipeline.fit(X, y)

# # Make predictions
# predictions = pipeline.predict(X_test)


Tried using AutoML H2O and accelerate them with the GPU to see if we could get any insights or improvements in our predictions, however, we were not able to make it run quickly in the GPU.

In [ ]:
import pandas as pd
from flaml import AutoML
from skrub import TableVectorizer


# Preprocess the dataset
X = df.drop(columns=['log_bike_count', 'bike_count'])
y = df['log_bike_count']

# Split the data into training and validation sets based on the last 10% of dates
validation_split_index = int(len(df) * 0.9)
X_train, X_val = X.iloc[:validation_split_index], X.iloc[validation_split_index:]
y_train, y_val = y.iloc[:validation_split_index], y.iloc[validation_split_index:]

# Initialize the TableVectorizer
vectorizer = TableVectorizer()

# Fit and transform the training data
X_train_transformed = vectorizer.fit_transform(X_train)
X_val_transformed = vectorizer.transform(X_val)



In [ ]:
import optuna
import joblib
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from ngboost import NGBRegressor
from h2o.automl import H2OAutoML
import h2o

# Initialize H2O
h2o.init()

# Dictionary to store the best parameters for each model
best_params = {}

# Define the objective function for Optuna
def objective(trial):
    # Model selection
    model_name = trial.suggest_categorical("model", ["RandomForest", "NGBoost", "H2OAutoML"])
    
    if model_name == "RandomForest":
        n_estimators = trial.suggest_int("n_estimators", 50, 500)
        max_depth = trial.suggest_int("max_depth", 2, 32)
        min_samples_split = trial.suggest_int("min_samples_split", 2, 20)
        min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 10)
        model = RandomForestRegressor(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=42,
        )
        model.fit(X_train_transformed, y_train)
        y_pred = model.predict(X_val_transformed)
    
    elif model_name == "NGBoost":
        learning_rate = trial.suggest_loguniform("learning_rate", 1e-4, 1e-1)
        n_estimators = trial.suggest_int("n_estimators", 50, 500)
        model = NGBRegressor(
            learning_rate=learning_rate,
            n_estimators=n_estimators,
            random_state=42,
        )
        model.fit(X_train_transformed, y_train)
        y_pred = model.predict(X_val_transformed)
    
    elif model_name == "H2OAutoML":
        # Convert datasets to H2O frames
        train = h2o.H2OFrame(pd.concat([X_train, y_train], axis=1))
        val = h2o.H2OFrame(pd.concat([X_val, y_val], axis=1))
        
        # Specify predictors and response column
        predictors = X_train.columns.tolist()
        response = "log_bike_count"  # Update with your target column name
        
        # Run H2O AutoML
        automl = H2OAutoML(max_models=10, seed=42, nfolds=3)
        automl.train(x=predictors, y=response, training_frame=train)
        
        # Predict on validation set
        y_pred = automl.leader.predict(val).as_data_frame()["predict"].values

    # Compute the Mean Squared Error (MSE)
    mse = mean_squared_error(y_val, y_pred)
    return mse

# Run Optuna optimization
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)

# Get the best trial and parameters
best_trial = study.best_trial
best_model_params = study.best_params
print("Best Trial:", best_trial)
print("Best Model Parameters:", best_model_params)

# Save the best model
model_name = best_model_params["model"]
if model_name == "H2OAutoML":
    # Save H2O AutoML model
    automl.leader.save_mojo(f"best_{model_name}.mojo")
    print(f"Best H2O AutoML model saved as 'best_{model_name}.mojo'")
else:
    # Save sklearn or NGBoost models
    joblib.dump(best_model, f"best_{model_name}.joblib")
    print(f"Best model saved as 'best_{model_name}.joblib'")

# Shut down H2O
h2o.shutdown(prompt=False)
